# Geokr

## Get config

In [ ]:
import json

with open('.config') as fp:
    config = json.load(fp)

## Flickr class

In [ ]:
import requests
import json

class Flickr:
    request_baseurl = 'https://api.flickr.com/services/rest/'
    jsonDecoder = json.JSONDecoder()
    
    def __init__(self, api_key):
        self.api_key = api_key

    def request(self, method, args):
        url = self.request_baseurl + '?api_key=' + self.api_key + '&format=json&method=' + method + '&' + '&'.join([k+'='+str(v) for k,v in args.iteritems()])
        return self.jsonDecoder.decode(requests.get(url).content[len("jsonFlickrApi("):-1])

    def get_photo_url(farm_id, server_id, photo_id):
        return "https://farm" + farm_id + ".staticflickr.com/" + server_id + "/" + photo_id + "_" + secret + ".jpg"


## Get recents photos

### Prepare requests

In [ ]:
flickr = Flickr(config['API_KEY'])
data = []
nb_page = 10
method = 'flickr.photos.getRecent'
args = {'per_page':500,'extras':'geo'}

### Send and parse requests

In [ ]:
for page in range(nb_page):
    args['page'] = page + 1
    data += flickr.request(method, args)["photos"]["photo"]

### Fetch results

In [ ]:
print "%d photos found" % len(data)
geotagged = []

for photo in data:
    if photo["longitude"] != 0 or photo["latitude"] != 0:
        geotagged.append(photo)

print "%d geotagged photos found" % len(geotagged)

## Get geotagged photos

### Prepare requests

In [ ]:
flickr = Flickr(config['API_KEY'])
data = []
nb_page = 8
method = 'flickr.photos.search'
args = {
    'has_geo': 1,
    'per_page': 250,
    'extras': 'geo,date_taken',
    'min_taken_date': '2015-01-01',
    'max_taken_date': '2016-01-01'
}

### Send and parse requests

In [ ]:
import json

for page in range(nb_page):
    args['page'] = page + 1
    data += flickr.request(method, args)["photos"]["photo"]

### Fetch results

In [ ]:
print "%d photos found" % len(data)

## KMeans

### Get the coordinates from data

In [ ]:
import numpy as np

coordinates = []

for photo in data:
    coordinates += [[photo["latitude"], photo["longitude"]]]

coordinates = np.array(coordinates, dtype=float)

### Run the algorithm

In [ ]:
from scipy.cluster.vq import kmeans2

k = 15
i = 500

centroids, closest_centroids = kmeans2(coordinates, k, iter=i)

print centroids

## Plot

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

for centroid in centroids:
    plt.scatter(centroid[0], centroid[1], c=np.random.rand(3,1))
    
plt.show()

In [ ]:
import folium
from IPython.display import HTML

def inline_map(m, width=800, height=600):
    """Takes a folium instance and embed HTML."""
    
    srcdoc = m._repr_html_().replace('"', '&quot;')
    embed = HTML('<iframe srcdoc="{}" '
                 'style="width: {}px; height: {}px; '
                 'border: none"></iframe>'.format(srcdoc, width, height))
    return embed


width, height = 800, 600
radars = folium.Map(location=[40, -122], zoom_start=1,
                    tiles='OpenStreetMap', width=width, height=height)

for centroid in centroids:
    folium.Marker([centroid[0], centroid[1]], ).add_to(radars) 



inline_map(radars)

### DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn import metrics

db = DBSCAN(eps=10,min_samples=40).fit(coordinates)

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(db.labels_)) - (1 if -1 in db.labels_ else 0)

print 'Estimated number of clusters: %d' % n_clusters_
print "Silhouette Coefficient: %0.3f" % metrics.silhouette_score(coordinates, db.labels_)

print "%d labels" % len(db.labels_)
print "%d coordinates" % len(coordinates)

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

colors = [np.random.rand(3,1) for i in range(n_clusters_+1)]

i = 0
for label in db.labels_:
    plt.scatter(coordinates[i][0], coordinates[i][1], c=colors[(label+1)])
    i += 1
    
plt.show()

## Convert centroids to GeoJSON

In [ ]:
import json

features = ""

for i in range(len(centroids)):
    geometry = {}
    geometry["type"] = "Point"
    geometry["coordinates"] = [centroids[i][1], centroids[i][0]]
    features += "{\"type\": \"Feature\", \"properties\": {}, \"geometry\": " + json.dumps(geometry) + "}" 
    if i < len(centroids) - 1:
        features += ", "
    
print "{\"type\" : \"FeatureCollection\", \"features\": [" + features +"]}"